In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
access_token = os.getenv('ACCESS_TOKEN')

In [5]:
import requests
from datetime import datetime
import pandas as pd

# Define variables
instrument_key = 'NSE_INDEX|Nifty Bank'
interval = '1minute'
from_date = '2024-11-15'
to_date = datetime.today().strftime('%Y-%m-%d')

# Ensure access_token is defined
access_token = "your_access_token_here"

# Construct URL
url = f"https://api.upstox.com/v2/historical-candle/{instrument_key}/{interval}/{to_date}/{from_date}"

# Headers including the access token
headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}',
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    chart_data = response.json()['data']['candles']
    
    # Convert to DataFrame
    df = pd.DataFrame(chart_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'other'])
    
    # Remove the last two columns and reverse the DataFrame
    df = df.iloc[:, :-2].iloc[::-1].reset_index(drop=True)
    
    # Rename columns
    df.columns = ['timestamp', 'open', 'high', 'low', 'close']
    
    # Convert the 'timestamp' column to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # for 5 min
    
    # # Set 'timestamp' as the index
    # df.set_index('timestamp', inplace=True)
    
    # # Resample the DataFrame
    # df = df.resample('5T').agg({
    #     'open': 'first',
    #     'high': 'max',
    #     'low': 'min',
    #     'close': 'last'
    # }).dropna()
    
    # # Reset the index
    # df.reset_index(inplace=True)

    # 5 min end
    
    # Create separate 'date' and 'time' columns
    df['date'] = df['timestamp'].dt.date
    df['time'] = df['timestamp'].dt.time
    
    # Drop the original 'timestamp' column if no longer needed
    df.drop(columns=['timestamp'], inplace=True)

    print(df)
else:
    print(f"Failed to fetch chart data. Status code: {response.status_code}, Error: {response.text}")

          open      high       low     close        date      time
0     50312.45  50344.10  50187.35  50273.50  2024-11-18  09:15:00
1     50265.60  50265.60  50196.55  50223.65  2024-11-18  09:16:00
2     50227.65  50228.10  50169.35  50182.10  2024-11-18  09:17:00
3     50181.70  50283.40  50172.90  50260.25  2024-11-18  09:18:00
4     50257.45  50261.70  50220.95  50258.45  2024-11-18  09:19:00
...        ...       ...       ...       ...         ...       ...
5995  53586.95  53591.85  53580.40  53586.55  2024-12-10  15:25:00
5996  53582.95  53601.65  53582.95  53599.25  2024-12-10  15:26:00
5997  53594.50  53610.40  53593.50  53602.25  2024-12-10  15:27:00
5998  53601.60  53623.05  53595.75  53607.75  2024-12-10  15:28:00
5999  53608.15  53624.05  53586.50  53611.10  2024-12-10  15:29:00

[6000 rows x 6 columns]


In [6]:
# Filter the DataFrame for the time range 09:15 to 10:00
filtered_df = df[(df['time'] >= pd.to_datetime('09:15:00').time()) & (df['time'] <= pd.to_datetime('15:30:00').time())]

print(filtered_df)




          open      high       low     close        date      time
0     50312.45  50344.10  50187.35  50273.50  2024-11-18  09:15:00
1     50265.60  50265.60  50196.55  50223.65  2024-11-18  09:16:00
2     50227.65  50228.10  50169.35  50182.10  2024-11-18  09:17:00
3     50181.70  50283.40  50172.90  50260.25  2024-11-18  09:18:00
4     50257.45  50261.70  50220.95  50258.45  2024-11-18  09:19:00
...        ...       ...       ...       ...         ...       ...
5995  53586.95  53591.85  53580.40  53586.55  2024-12-10  15:25:00
5996  53582.95  53601.65  53582.95  53599.25  2024-12-10  15:26:00
5997  53594.50  53610.40  53593.50  53602.25  2024-12-10  15:27:00
5998  53601.60  53623.05  53595.75  53607.75  2024-12-10  15:28:00
5999  53608.15  53624.05  53586.50  53611.10  2024-12-10  15:29:00

[6000 rows x 6 columns]


In [7]:
import plotly.graph_objects as go

# Group the DataFrame by date
grouped = df.groupby('date')

# Create a list to store the individual plots
plots = []

# Iterate over each group (each day)
for date, group in grouped:
    # Convert 'time' column to datetime
    group['time'] = pd.to_datetime(group['time'], format='%H:%M:%S')
    
    # Calculate the high-low difference for the day
    high_low_diff = group['high'].max() - group['low'].min()
    
    # Create the candlestick plot for the day
    fig = go.Figure(data=[go.Candlestick(
        x=group['time'],
        open=group['open'],
        high=group['high'],
        low=group['low'],
        close=group['close']
    )])
    
    # Update layout for the plot
    fig.update_layout(
        title=f'Date: {date.strftime("%d-%m-%Y")} | Diff: {high_low_diff:.2f}',
        xaxis_title='Time',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        xaxis=dict(
            tickformat='%H:%M',
            tickmode='linear',
            dtick=1800000  # 30 minutes in milliseconds
        ),
        width=1400,
        height=500,
        template='plotly_dark',  # Use a dark theme for better contrast
        showlegend=False
    )
    
    # Append the plot to the list
    plots.append(fig)

# Display all the plots in one frame
for fig in plots:
    fig.show()
